In [1]:
import time
# Startzeitpunkt erfassen
start_time = time.time()

import pandas as pd
import numpy as np
import yahooquery as yq
import datetime as dt
from dateutil.relativedelta import relativedelta
import os
import yfinance as yf
import tkinter as tk
from tkinter import filedialog
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter


Inputdaten einlesen

In [2]:
# Dialogfenster erzeugen
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
root.lift()

# Dateiname Inputtabelle
input_data = filedialog.askopenfilename(title = "Watchliste öffnen", initialfile="Watchliste", filetypes=(("Excel-Dateien", ".xlsx"), ("alle Dateitypen", ".*")))

In [3]:
# Einlesen der Excel-Datei
input_namen = {"Asset": str,
               "Symbol (Yahoo)": str,
               "ISIN": str,
               "Benchmarkindex": str,
               "Symbol Benchmark": str,
}
df_input_data = pd.read_excel(input_data, names= input_namen.keys(), dtype= input_namen)

In [4]:
stocks =[]
benchmarks = []
df_symbols_mapping = ()

list_assets = df_input_data["Symbol (Yahoo)"].tolist()
list_benchmarks = df_input_data["Symbol Benchmark"].tolist()

for asset in list_assets:
    stocks.append(yq.Ticker(asset))

for benchmark in list_benchmarks:
    benchmarks.append(yq.Ticker(benchmark))

df_symbols_mapping = pd.DataFrame({"symbol": list_assets, "benchmark": list_benchmarks})

In [5]:
today = dt.date.today()

def get_previous_business_day(date):
    # datum um "date" Monate zurückrechnen, wobei z.B. 3 Monate vom 31.05. zurück -> 28.02. ergibt
    previous_date = today + relativedelta(months=-date)
    # Überprüfen, ob der gegebene Tag ein Werktag ist
    # (Montag = 0, Sonntag = 6)
    if previous_date.weekday() > 4:
        delta = previous_date.weekday() - 4
        previous_date -= dt.timedelta(days=delta)
    return previous_date

# Liste mit Heutigem Datum
actual_date = [today, "heute"]

# Das Datum von vor 1 Monat ermitteln
previous_1_month = [get_previous_business_day(1), "vor 1 Monat"]

# Das Datum von vor 2 Monaten ermitteln
previous_2_months = [get_previous_business_day(2), "vor 2 Monaten"]

# Das Datum von vor 3 Monaten ermitteln
previous_3_months = [get_previous_business_day(3), "vor 3 Monaten"]

# Das Datum von vor 6 Monaten ermitteln
previous_6_months = [get_previous_business_day(6), "vor 6 Monaten"]

# Das Datum von vor 1 Jahr ermitteln
previous_1_year = [get_previous_business_day(12), "vor 1 Jahr"]

df_previous_days =  pd.DataFrame([actual_date, previous_1_month, previous_2_months, previous_3_months, previous_6_months, previous_1_year], columns=["Dates", "Description"])
df_previous_days["Dates"] = pd.to_datetime(df_previous_days["Dates"]).dt.strftime('%Y-%m-%d')

In [6]:
# Ermitteln ob ein Tag ein Börsentag ist
def is_business_day(date):
    # Überprüfen, ob der gegebene Tag ein Werktag ist (Montag = 0, Sonntag = 6)
    if date.weekday() >= 5:
        return False
    return True

# Ermitteln des nächsten Börsentages, falls für den Earnings Announcement- oder Post-Announcementtag kein Kurs gefunden wird
def get_next_trading_day(date):
    if date in df_prices["date"].values:
        return date
    else:
        next_trading_day = df_prices[df_prices["date"] > date]["date"].min()
        return next_trading_day

Abfrage der Daten von Yahoo Finance

In [7]:
# Termine der vergangenen Bekanntgaben von QUartalszahlen ermitteln
# df_earnings_latest = nur die aktuellsten QUartalszahlen je Ticker
# df_earnigns = die vergangenen Quartalszahlen je Ticker (hier nicht benötigt)

df_earnings_data = pd.DataFrame()
df_earnings = pd.DataFrame()

for asset in list_assets:
    df_earnings_data = yf.Ticker(asset).get_earnings_dates()
    df_earnings_data["symbol"] = asset
    df_earnings_data["Description"] = "earnings announcement"
    df_earnings = pd.concat([df_earnings, df_earnings_data])
    time.sleep(1.3)

df_earnings = df_earnings.reset_index()
# Umwandlung des Datums in die Form "2024-01-22"
df_earnings["Dates"] = pd.to_datetime(df_earnings["Earnings Date"], utc= True).dt.strftime('%Y-%m-%d')
df_earnings.drop(["Earnings Date"], axis= 1)

df_earnings_latest = df_earnings[df_earnings["Dates"] < str(today)].sort_values("Dates", ascending=False)
df_earnings_latest = df_earnings_latest.groupby("symbol").first().reset_index()
df_earnings_latest = df_earnings_latest.set_index("Dates", drop=True)

# Series umwandeln in ein Data Frame inkl. einer Spalte mit Beschreibung
df_earnings_latest.loc["Dates"] = pd.to_datetime(df_earnings["Dates"], format="%Y-%m-%d")
df_earnings = df_earnings.dropna(subset=["Reported EPS"])

In [8]:
# Data Frame mit den Datumsangaben inkl. Beschreibung der Daten zusammenfügen
df_dates = pd.concat([df_previous_days, df_earnings])
df_dates = df_dates.reset_index(drop=True)
df_dates = df_dates[["symbol", "Dates", "Description"]]
df_dates = df_dates.dropna(subset=["Dates"])
df_dates = df_dates.sort_values(by=("Dates"), ascending=False)

In [9]:
# Kursabfrage des letzten Jahres
df_kurse = pd.DataFrame()
df_benchmarks = pd.DataFrame()
df_prices = pd.DataFrame()

for stock in stocks:
    stock_data = stock.history(period="1y", interval="1d")
    df_kurse = pd.concat([df_kurse, stock_data])
    time.sleep(1.3)

for benchmark in benchmarks:
    benchmark_data = benchmark.history(period="1y", interval="1d")
    df_benchmarks = pd.concat([df_benchmarks, benchmark_data])
    time.sleep(1.3)

df_kurse = df_kurse.reset_index()
df_benchmarks = df_benchmarks.reset_index()
df_kurse["date"] = pd.to_datetime(df_kurse["date"], utc=True).dt.strftime("%Y-%m-%d")
df_benchmarks["date"] = pd.to_datetime(df_benchmarks["date"], utc=True).dt.strftime("%Y-%m-%d")

# Unnötige Spalten entfernen
df_kurse_new = df_kurse[["symbol", "date", "close"]]
df_benchmarks_new = df_benchmarks[["symbol", "date", "close"]]
df_kurse_new = df_kurse_new.rename(columns={"close": "previous close"})
df_benchmarks_new = df_benchmarks_new.rename(columns={"symbol": "benchmark", "close": "previous close"})

# Data Frames zusammenführen (zuerst via Mappingtabelle)
df_prices = pd.merge(df_kurse_new, df_symbols_mapping, how="left", on="symbol")
df_prices = pd.merge(df_prices, df_benchmarks_new, how="left", on=("date", "benchmark"), suffixes=("", "_benchmark"))
df_prices["date"] = pd.to_datetime(df_prices["date"]).dt.strftime("%Y-%m-%d")
df_prices = df_prices.drop_duplicates()

In [10]:
# Fundamentalkennzahlen der letzten 3 Geschäftsjahre abfragen
df_fundamentals = pd.DataFrame()

for stock in stocks:
    df_fundamentals = pd.concat([df_fundamentals, (stock.get_financial_data(types=["BasicEPS", 
                                                                             "DilutedEPS", 
                                                                             "EBIT", 
                                                                             "TotalRevenue", 
                                                                             "MarketCap", 
                                                                             "EnterpriseValue", 
                                                                             "CommonStockEquity", 
                                                                             "NetIncome", 
                                                                             "TotalAssets", 
                                                                             "priceToBook", 
                                                                             "lastFiscalYearEnd"]))
                                                                             ])
    time.sleep(1.3)

df_fundamentals = df_fundamentals.reset_index()

In [11]:
# Analystenmeinungen
dict_analysts = {}
for stock in stocks:
    tmp_dict_var = pd.DataFrame(stock.financial_data)
    dict_analysts = {**dict_analysts, **tmp_dict_var}
    time.sleep(1.3)

df_analysts = pd.DataFrame(dict_analysts)
df_analysts.set_index(keys=df_analysts.columns[0])
df_analysts = df_analysts.loc[["recommendationMean", "numberOfAnalystOpinions", "returnOnEquity"],:]
df_analysts = df_analysts.transpose()

In [12]:
# Aktuelle Fundamentalkennzahlen
dict_fundamentals_current = {}
tmp_dict_var = {}
for stock in stocks:
    tmp_dict_var = pd.DataFrame(stock.summary_detail)
    dict_fundamentals_current = {**dict_fundamentals_current, **tmp_dict_var}
    time.sleep(1.3)

df_fundamentals_current = pd.DataFrame(dict_fundamentals_current)
df_fundamentals_current.set_index(keys=df_fundamentals_current.columns[0])
df_fundamentals_current = df_fundamentals_current.loc[["trailingPE", "forwardPE","marketCap"],:]

In [13]:
# Künftige Quartalszahlen (Schätzungen)
# Alle Zahlen abfragen (nested/verschachteltes Dictionary)
dict_earnings_future = {}
tmp_dict_var = {}
for stock in stocks:
    tmp_dict_var = pd.DataFrame(stock.earnings_trend)
    dict_earnings_future = {**dict_earnings_future, **tmp_dict_var}
    time.sleep(1.3)

# Erzeugen eines neuen Dictionarys, das gefiltert nach "trend" (so ist die Bezeichnung des Keys) sowie nach den Perioden "0y" (aktuelles Jahr) und "+1y" (nächstes Jahr), sowie nur die benötigten Keys "epsTrend" und "epsRevisions" enthält
dict_filtered_data = []
for key, value in dict_earnings_future.items():
    for trend in value["trend"]:
        if trend["period"] in ["0y", "+1y"]:
            dict_filtered_data.append({
                "symbol": key,
                "period": trend["period"],
                "epsTrend": trend["epsTrend"],
                "epsRevisions": trend["epsRevisions"]
            })

# Erzeugen des DataFrames
df_earnings_future = pd.DataFrame(dict_filtered_data)
df_earnings_future = pd.concat([df_earnings_future.drop(["epsTrend"], axis=1), df_earnings_future["epsTrend"].apply(pd.Series).add_prefix("epsTrend_")], axis=1)
df_earnings_future = pd.concat([df_earnings_future.drop(["epsRevisions"], axis=1), df_earnings_future["epsRevisions"].apply(pd.Series).add_prefix("epsRevisions_")], axis=1)

In [14]:
# Anlegen des Ergebnis-Dataframes mit den einzelnen Assets als Zeilenindex
df_results = []
df_results = pd.DataFrame(index= list_assets)

In [15]:
# Punkte für RoE
# Wenn >20% dann +1, wenn <10% dann -1, sonst 0

# Funktion zur Bewertung:
def calculate_roe(calc):
    if isinstance(calc, dict):  # Überprüfen, ob der Wert ein Dictionary ist
        if "value" in calc:
            calc = calc["value"]
        else:
            return "keine Daten"  # Wenn "value" nicht im Dictionary vorhanden ist, gib 0 zurück
    if calc is None:
        return "keine Daten"
    elif calc > 0.2:
        return 1
    elif calc >= 0.1 and calc <= 0.2:
        return 0
    elif calc < 0.1:
        return -1

df_results["RoE"] = df_analysts["returnOnEquity"].apply(calculate_roe)
df_analysts["Score RoE"] = df_analysts["returnOnEquity"].apply(calculate_roe)

In [16]:
# Punkte für EBIT-Marge
# Wenn >12% dann +1, wenn <6% dann -1, sonst 0

# Funktion zur Bewertung:
def calculate_ebit_margin(row):
    ebit = row["EBIT"]
    revenue = row["TotalRevenue"]
    if (ebit is None) | (revenue is None):
        return "keine Daten"
    elif (ebit/revenue) > 0.12:
        return 1
    elif (ebit/revenue) >= 0.06 and (ebit/revenue) <= 0.12:
        return 0
    elif (ebit/revenue) < 0.06:
       return -1

# Sortiere das Dataframe Fundamentals nach Datum, um im nächsten Schritt jeweils den aktuellsten vollständigen Eintrag zu wählen
df_ebit_sorted = []
df_ebit_sorted = df_fundamentals.sort_values("asOfDate", ascending=False)

# Gruppiere das Dataframe nach dem Symbol und wähle die erste Zeile für jedes Symbol aus
df_ebit_latest = df_ebit_sorted.groupby("symbol").first().reset_index()
df_ebit_latest = df_ebit_latest.set_index("symbol", drop=True)

df_results["EBIT margin"] = df_ebit_latest.apply(calculate_ebit_margin, axis=1)
df_ebit_latest["Score EBIT margin"] = df_ebit_latest.apply(calculate_ebit_margin, axis=1)

In [17]:
# Punkte für EK-Quote
# Wenn >25% dann +1, wenn <15% dann -1, sonst 0

# Funktion zur Bewertung:
def calculate_equity_ratio(row):
    common_stock = row["CommonStockEquity"]
    assets = row["TotalAssets"]
    if (common_stock is None) | (assets is None):
        return "keine Daten"
    elif (common_stock/assets) > 0.25:
        return 1
    elif (common_stock/assets) >= 0.15 and (common_stock/assets) <= 0.25:
        return 0
    elif (common_stock/assets) < 0.15:
       return -1

# Sortiere das Dataframe Fundamentals nach Datum, um im nächsten Schritt jeweils den aktuellsten vollständigen Eintrag zu wählen
df_equity_ratio_sorted = []
df_equity_ratio_sorted = df_fundamentals.sort_values("asOfDate", ascending=False)

# Gruppiere das Dataframe nach dem Symbol und wähle die erste Zeile für jedes Symbol aus
df_equity_ratio_latest = df_equity_ratio_sorted.groupby("symbol").first().reset_index()
df_equity_ratio_latest = df_equity_ratio_latest.set_index("symbol", drop=True)

df_results["equity ratio"] = df_ebit_latest.apply(calculate_equity_ratio, axis=1)
df_ebit_latest["Score equity ratio"] = df_ebit_latest.apply(calculate_equity_ratio, axis=1)

In [18]:
# Punkte für 5-Jahres-KGV
# letzte 3 Jahre, aktuelles Geschäftsjahr, nächstes Geschäftsjahr
# Wenn <12 dann +1, wenn >16 oder <0 dann -1, sonst 0
df_pe_ratio_5 = pd.DataFrame()

df_pe_ratio_5 = df_fundamentals[df_fundamentals["periodType"] == "12M"][["symbol", "asOfDate", "DilutedEPS"]]
df_pe_ratio_5 = df_pe_ratio_5.sort_values("asOfDate", ascending=False)
df_pe_5_agg = df_pe_ratio_5.groupby("symbol").head(3)

# Dataframe mit den prognostizierten EPS des aktuellen/kommenden Jahres:
df_pe_future = df_earnings_future[["symbol", "period", "epsTrend_current"]]
df_pe_future = df_pe_future.rename(columns={"period": "asOfDate", "epsTrend_current": "DilutedEPS"})

# Dataframes zusammenführen und EPS addieren
df_pe_5_agg = pd.concat([df_pe_5_agg, df_pe_future])
df_pe_5_agg = df_pe_5_agg.groupby("symbol").agg({"DilutedEPS": "sum", "symbol": "count"}).rename(columns={"symbol": "Count"})

# Ermitteln des aktuellsten Kurses
df_prices_pe = df_kurse
df_prices_pe = df_prices_pe.sort_values("date", ascending=False)
df_prices_pe = df_prices_pe.groupby("symbol").first().reset_index()
df_prices_pe = df_prices_pe.set_index("symbol", drop=True)
df_prices_pe = df_prices_pe[["date", "close"]]
df_prices_pe["date"] = pd.to_datetime(df_prices_pe["date"]).dt.strftime('%Y-%m-%d')
df_prices_pe = df_prices_pe.rename(columns={"close": "previous close"})

df_pe_5_results = pd.merge(df_pe_5_agg, df_prices_pe, left_index=True, right_index=True, how="inner")

def calculate_pe_ratio(row):
    price = row["previous close"]
    earnings = row["DilutedEPS"]
    count = row["Count"]
    if count == 0:
        return "keine Daten"
    elif ((price * count)/earnings) < 12:
        return 1
    elif ((price * count)/earnings) >= 12 and ((price * count)/earnings) <= 16:
        return 0
    elif ((price * count)/earnings) > 16:
        return -1
    elif ((price * count)/earnings) < 0:
        return -1
    
df_results["pe_ratio_5yr"] = df_pe_5_results.apply(calculate_pe_ratio, axis=1)
df_pe_5_results["Score pe_ratio_5yr"] = df_pe_5_results.apply(calculate_pe_ratio, axis=1)

In [19]:
# Punkte für aktuelles KGV
# Wenn <12 dann +1, wenn >16 oder <0 dann -1, sonst 0
df_pe_ratio_current = pd.DataFrame()
df_pe_future = pd.DataFrame()
df_prices_pe_current = pd.DataFrame()
df_pe_current_results = pd.DataFrame()

df_pe_ratio_current = df_fundamentals[df_fundamentals["periodType"] == "12M"][["symbol", "asOfDate", "DilutedEPS"]]
df_pe_ratio_current = df_pe_ratio_current.sort_values("asOfDate", ascending=False)
df_pe_ratio_current = df_pe_ratio_current.groupby("symbol").first()
df_pe_ratio_current = df_pe_ratio_current.rename(columns={"DilutedEPS": "EPS_last"})

# Dataframe mit den prognostizierten EPS des aktuellen/kommenden Jahres:
df_pe_current_future = df_earnings_future[df_earnings_future["period"] == "0y"][["symbol", "period", "epsTrend_current"]]
df_pe_current_future = df_pe_current_future.rename(columns={"period": "asOfDate", "epsTrend_current": "DilutedEPS"})
df_pe_current_future = df_pe_current_future.set_index(df_pe_current_future["symbol"])
df_pe_current_future = df_pe_current_future.rename(columns={"DilutedEPS": "EPS_current"})

# Ermitteln des aktuellsten Kurses
df_prices_pe_current = df_kurse
df_prices_pe_current = df_prices_pe_current.sort_values("date", ascending=False)
df_prices_pe_current = df_prices_pe_current.groupby("symbol").first().reset_index()
df_prices_pe_current = df_prices_pe_current.set_index("symbol", drop=True)
df_prices_pe_current = df_prices_pe_current[["date", "close"]]
df_prices_pe_current["date"] = pd.to_datetime(df_prices_pe_current["date"]).dt.strftime('%Y-%m-%d')
df_prices_pe_current = df_prices_pe_current.rename(columns={"close": "previous close"})

# Data Frames zusammenführen für die Funktion
df_pe_current_results = pd.merge(df_pe_ratio_current, df_pe_current_future, left_index=True, right_index=True, how="inner")
df_pe_current_results = pd.merge(df_pe_current_results, df_prices_pe_current, left_index=True, right_index=True, how="inner")
df_pe_current_results = df_pe_current_results.set_index(df_pe_current_results["symbol"])

def calculate_pe_ratio_current(row):
    price = row["previous close"]
    earnings_current = row["EPS_current"]
    earnings_past = row["EPS_last"]
    if (earnings_current is None) | (earnings_current == 0) | (earnings_current == ""):
        if (earnings_past is None) | (earnings_past == 0) | (earnings_past == ""):
            return "keine Daten"
        elif (price/earnings_past) < 12:
            return 1
        elif (price/earnings_past) >= 12 and (price/earnings_past) <= 16:
            return 0
        elif (price/earnings_past) > 16:
            return -1
    elif (price/earnings_current) < 12:
        return 1
    elif (price/earnings_current) >= 12 and (price/earnings_current) <= 16:
        return 0
    elif (price/earnings_current) > 16:
        return -1
    elif (price/earnings_current) < 0:
        return -1
    
df_results["pe_ratio_current"] = df_pe_current_results.apply(calculate_pe_ratio_current, axis=1)
df_pe_current_results["Score pe_ratio_current"] = df_pe_current_results.apply(calculate_pe_ratio_current, axis=1)

In [20]:
# Punkte für Analystenmeinungen
# Wenn Anzahl <5 und Meinung <2 dann 1, 
# wenn Anzahl <5 und Meinung zwischen 2 und 4 dann 0,
# wenn Anzahl <5 und Meinung >4 dann -1,
# wenn Anzahl >5 und Meinung >4 dann 1,
# wenn Anzahl >5 und Meinung zwischen 2 und 4 dann 0
# wenn Anzahl >5 und Meinung <2 dann -1

def calculate_analysts(row):
    valuation = row["recommendationMean"]
    count = row["numberOfAnalystOpinions"]
    if (count is None) | (valuation is None):
        return "keine Daten"
    elif (count < 5) and (valuation <2):
        return 1
    elif (count < 5) and (valuation >= 2) and (valuation <=4):
        return 0
    elif (count < 5) and (valuation > 4):
        return -1
    elif (count > 5) and (valuation > 4):
        return 1
    elif (count > 5) and (valuation >= 2) and (valuation <= 4):
        return 0
    elif (count > 5) and (valuation < 2):
        return -1
        
df_results["analysts_opinion"] = df_analysts.apply(calculate_analysts, axis=1)
df_analysts["Score analysts_opinion"] = df_analysts.apply(calculate_analysts, axis=1)

In [21]:
# Punkte für Quartalszahlen
# Punkte Quartalszahlen: +1 wenn Differenz Entwicklung (nach letzten Earnings) >1%, -1 wenn Differenz Entwicklung <-1%, sonst 0
# Entwicklung Kurs letzte Earnings ((Kurs nach Earnings - Kurs Vortag) / Kurs Vortag)
# Entwicklung Kurs Benchmark (analog dazu)
# Differenz Entwicklung (Entw. Kurs - Entw. Benchmark)

df_history_announcements = pd.DataFrame()
df_latest_announcement = pd.DataFrame()

# Ermittlung der Daten der letzten Quartalszahlen
df_latest_announcement = df_dates
df_latest_announcement = df_latest_announcement.sort_values("Dates", ascending=False)
df_latest_announcement = df_latest_announcement.groupby("symbol").first()

df_prices["date"] = pd.to_datetime(df_prices["date"])
df_latest_announcement["Dates"] = pd.to_datetime(df_latest_announcement["Dates"])
df_latest_announcement["announcement_next_trading_day"] = df_latest_announcement["Dates"].apply(get_next_trading_day)

df_latest_announcement["post date"] = pd.to_datetime(df_latest_announcement["Dates"]) + pd.DateOffset(days = 1)
df_latest_announcement["post date"] = df_latest_announcement["post date"].apply(get_next_trading_day)
df_latest_announcement["post date"] = pd.to_datetime(df_latest_announcement["post date"])
df_latest_announcement = df_latest_announcement.rename(columns={"announcement_next_trading_day": "date"})
df_latest_announcement.drop(columns=["Dates"])

# Anfügen der Kurse von Asset und Benchmark zu Datum der letzten Earnings
df_history_announcements = pd.merge(df_latest_announcement, df_prices, how="left", left_on=["symbol", "date"], right_on=["symbol", "date"])
df_history_announcements = pd.merge(df_history_announcements, df_prices, how="left", left_on=["symbol", "post date"], right_on=["symbol", "date"], suffixes=("", "_post"))
df_history_announcements = df_history_announcements.drop_duplicates()
df_history_announcements = df_history_announcements.set_index("symbol", drop=False)
df_history_announcements = df_history_announcements[~df_history_announcements.index.duplicated(keep='first')]

# Bewertungsfunktion
def calculate_eps_reaction(row):
    symbol = row["symbol"]
    benchmark = row["benchmark"]
    benchmark_check = row["benchmark_post"]
    date_post = row["post date"]
    date_check = row["date_post"]
    date_announcement = row["date"]
    price_asset = row["previous close"]
    price_benchmark = row["previous close_benchmark"]
    price_asset_post = row["previous close_post"]
    price_benchmark_post = row["previous close_benchmark_post"]
    reaction_asset = (price_asset_post - price_asset) / price_asset
    reaction_benchmark = (price_benchmark_post - price_benchmark) / price_benchmark
    delta_reaction = reaction_asset - reaction_benchmark
    if (symbol is None) | (benchmark is None) | (date_post is None) | (date_announcement is None) | (benchmark != benchmark_check) | (date_post != date_check) | (date_post <= date_announcement) | (delta_reaction is None):
        return "keine Daten"
    elif delta_reaction > 0.01:
        return 1
    elif (delta_reaction <= 0.01) and (delta_reaction >= -0.01):
        return 0
    elif delta_reaction < -0.01:
        return -1

df_results["earnings reaction"] = df_history_announcements.apply(calculate_eps_reaction, axis=1)
df_history_announcements["Score earnings reaction"] = df_history_announcements.apply(calculate_eps_reaction, axis=1)


In [22]:
# Punkte für Revision der Ergebnisse
# Revision EPS AJ (Delta EPS zu vor 4 Wochen, falls möglich, ansonsten nimm den Wert von vor 2 Wochen) geteilt durch den Wert vor 4 bzw. 2 Wochen
# Revision EPS NJ (analog dazu)
# Punkte Revision EPS (für beide Quellen jeweils separat): wenn entweder Revision EPS AJ oder NJ leer ist, nur das andere betrachten: 
# +1 wenn Revision EPS NJ >5%, -1 wenn Revision EPS <-5%. Wenn beide gefüllt sind: Punkte der beiden saldieren, +2 bzw. -2 werden zu +1 bzw. -1. Sonst 0

df_revisions = pd.DataFrame()
df_revisions_0y = pd.DataFrame()
df_revisions_1y = pd.DataFrame()
df_revisions_tmp = pd.DataFrame()

# Daten abfragen und in 2 separate Dataframes (aktuelles Jahr und kommendes Jahr) einfügen
df_revisions = df_earnings_future[["symbol", "period", "epsTrend_current", "epsTrend_7daysAgo", "epsTrend_30daysAgo"]]
df_revisions_0y = df_revisions[df_revisions["period"] == "0y"]
df_revisions_1y = df_revisions[df_revisions["period"] == "+1y"]

df_revisions_0y = df_revisions_0y.set_index("symbol")
df_revisions_1y = df_revisions_1y.set_index("symbol")

def calculate_revisions(row):
    period = row["period"]
    epsToday = row["epsTrend_current"]
    epsOld = row["epsTrend_30daysAgo"]
    if (epsOld == 0) | (epsOld == ""):
        return "Division durch Null (EPS alt = Null)"
    elif (period != "0y" and period != "+1y"):
        return "keine Daten"
    elif ((epsToday - epsOld) / epsOld) > 0.05:
        return 1
    elif (((epsToday - epsOld) / epsOld) <= 0.05) and (((epsToday - epsOld) / epsOld) >= -0.05):
        return 0
    elif ((epsToday - epsOld) / epsOld) < -0.05:
        return -1

def calculate_revisions_alternative(row):
    period = row["period"]
    epsToday = row["epsTrend_current"]
    epsOld = row["epsTrend_7daysAgo"]
    if (epsOld == 0) | (epsOld == ""):
        return "Division durch Null (EPS alt = Null)"
    elif (period != "0y" and period != "+1y"):
        return "keine Daten"
    elif ((epsToday - epsOld) / epsOld) > 0.05:
        return 1
    elif (((epsToday - epsOld) / epsOld) <= 0.05) and (((epsToday - epsOld) / epsOld) >= -0.05):
        return 0
    elif ((epsToday - epsOld) / epsOld) < -0.05:
        return -1

# Punkte aufaddieren für aktuelles und für nächstes Jahr
df_revisions_tmp["Rev_0y"] = df_revisions_0y.apply(calculate_revisions, axis=1)
df_revisions_tmp["Rev_0y_alt"] = df_revisions_0y.apply(calculate_revisions_alternative, axis=1)
df_revisions_tmp["Rev_1y"] = df_revisions_1y.apply(calculate_revisions, axis=1)
df_revisions_tmp["Rev_1y_alt"] = df_revisions_1y.apply(calculate_revisions_alternative, axis=1)

# Bewerten
def add_rev_values(row):
    rev_0y = row["Rev_0y"]
    rev_1y = row["Rev_1y"]
    rev_0y_alt = row["Rev_0y_alt"]
    rev_1y_alt = row["Rev_1y_alt"]
    if rev_0y is None or pd.isnull(rev_0y) or pd.isna(rev_0y) or rev_0y == "":
        if rev_1y is None or pd.isnull(rev_1y) or pd.isna(rev_1y) or rev_1y == "":
            if rev_0y_alt is None or pd.isnull(rev_0y_alt) or pd.isna(rev_0y_alt) or rev_0y_alt == "":
                if rev_1y_alt is None or pd.isnull(rev_1y_alt) or pd.isna(rev_1y_alt) or rev_1y_alt == "":
                    result = "keine Daten"
                else:
                    result = rev_1y_alt
            else:
                result = rev_0y_alt
        else:
            result = rev_1y
    else:
        result = rev_0y         
    if result == 2:
        result = 1
    elif result == -2:
        result = -1 
    return result

df_revisions_tmp["earnings revisions"] = df_revisions_tmp.apply(add_rev_values, axis=1)

df_results = pd.merge(df_results, df_revisions_tmp["earnings revisions"], how="left", left_index=True, right_index=True,)
df_revisions = df_revisions_tmp

In [23]:
# Punkte 6-Monats-Entwicklung
# Punkte 6 Monate: +1 wenn relative Performance 6 Monate >5%, -1 wenn relative Performance 6 Monate <-5%, sonst 0
# Entwicklung Kurs Asset und Kurs Benchmark seit 6 Monaten ((letzter Kurs - Kurs vor 6 Monaten) / Kurs vor 6 Monaten)
# Differenz Entwicklung (Entw. Kurs - Entw. Benchmark)

df_6months_ago = pd.DataFrame()
df_latest_prices = pd.DataFrame()

# Ermittlung der aktuellsten Kurse
df_latest_prices = df_prices
df_latest_prices = df_latest_prices.sort_values("date", ascending=False)
df_latest_prices = df_latest_prices.groupby("symbol").first()
df_latest_prices["date"] = pd.to_datetime(df_latest_prices["date"]).dt.strftime('%Y-%m-%d')

# Ermittlung der Kurse vor 6 Monaten (bzw. am darauffolgenden nächsten verfügbaren Kurs)
df_6months_ago = df_prices
date_6months = df_previous_days.loc[df_previous_days["Description"] == "vor 6 Monaten", "Dates"].values[0]
date_6months = pd.to_datetime(date_6months).strftime('%Y-%m-%d')
df_6months_ago = df_6months_ago[df_6months_ago["date"] == date_6months]
# Überprüfung, ob Eintrag am vorliegenden Datum vorhanden ist
while df_6months_ago.empty:
    # Einen Tag zurückgehen
    date_6months -= dt.timedelta(days=1)
    df_6months_ago = df_prices[df_prices["date"] == date_6months]

df_6months_ago = df_6months_ago.drop_duplicates()
df_6months_ago = df_6months_ago.set_index("symbol")

df_6_months_comparison = pd.merge(df_6months_ago, df_latest_prices, how="left", left_index=True, right_index=True, suffixes=("_6months","_current"))

# Bewertungsfunktion
def calculate_6_months_performance(row):
    benchmark = row["benchmark_6months"]
    benchmark_check = row["benchmark_current"]
    date = row["date_6months"]
    date_check = row["date_current"]
    price_asset_old = row["previous close_6months"]
    price_benchmark_old = row["previous close_benchmark_6months"]
    price_asset_new = row["previous close_current"]
    price_benchmark_new = row["previous close_benchmark_current"]
    performance_asset = (price_asset_new - price_asset_old) / price_asset_old
    performance_benchmark = (price_benchmark_new - price_benchmark_old) / price_benchmark_old
    delta_performance = performance_asset - performance_benchmark
    if (date is None) | (price_asset_new is None) | (price_asset_new == 0) | (price_benchmark_new is None) | (price_benchmark_new == 0) | (date_check is None) | (benchmark is None) | (benchmark != benchmark_check) | (date >= pd.to_datetime(date_check)) | (delta_performance is None):
        return "keine Daten"
    elif delta_performance > 0.05:
        return 1
    elif (delta_performance <= 0.05) and (delta_performance >= -0.05):
        return 0
    elif delta_performance < -0.05:
        return -1

df_results["6 months performance"] = df_6_months_comparison.apply(calculate_6_months_performance, axis=1)
df_6_months_comparison["Score 6 months performance"] = df_6_months_comparison.apply(calculate_6_months_performance, axis=1)

In [24]:
# Punkte 1-Jahres-Entwicklung
# Punkte 1 Jahr: +1 wenn relative Performance 1 Jahr >5%, -1 wenn relative Performance 1 Jahr <-5%, sonst 0
# Entwicklung Kurs Asset und Kurs Benchmark seit 1 Jahr ((letzter Kurs - Kurs vor 1 Jahr) / Kurs vor 1 Jahr)
# Differenz Entwicklung (Entw. Kurs - Entw. Benchmark)

df_1year_ago = pd.DataFrame()
df_latest_prices = pd.DataFrame()

# Ermittlung der aktuellsten Kurse
df_latest_prices = df_prices
df_latest_prices = df_latest_prices.sort_values("date", ascending=False)
df_latest_prices = df_latest_prices.groupby("symbol").first()
df_latest_prices["date"] = pd.to_datetime(df_latest_prices["date"])

# Ermittlung der Kurse vor 6 Monaten (bzw. am darauffolgenden nächsten verfügbaren Kurs)
df_1year_ago = df_prices
date_1year = df_previous_days.loc[df_previous_days["Description"] == "vor 1 Jahr", "Dates"].values[0]
date_1year = pd.to_datetime(date_1year)
df_1year_ago = df_1year_ago[df_1year_ago["date"] == date_1year]
# Überprüfung, ob Eintrag am vorliegenden Datum vorhanden ist
while df_1year_ago.empty:
    # Einen Tag zurückgehen
    date_1year -= dt.timedelta(days=1)
    df_1year_ago = df_prices[df_prices["date"] == date_1year]

df_1year_ago = df_1year_ago.drop_duplicates()
df_1year_ago = df_1year_ago.set_index("symbol")

df_1_year_comparison = pd.merge(df_1year_ago, df_latest_prices, how="left", left_index=True, right_index=True, suffixes=("_1year","_current"))

# Bewertungsfunktion
def calculate_1_year_performance(row):
    benchmark = row["benchmark_1year"]
    benchmark_check = row["benchmark_current"]
    date = row["date_1year"]
    date_check = row["date_current"]
    price_asset_old = row["previous close_1year"]
    price_benchmark_old = row["previous close_benchmark_1year"]
    price_asset_new = row["previous close_current"]
    price_benchmark_new = row["previous close_benchmark_current"]
    performance_asset = (price_asset_new - price_asset_old) / price_asset_old
    performance_benchmark = (price_benchmark_new - price_benchmark_old) / price_benchmark_old
    delta_performance = performance_asset - performance_benchmark
    if (date is None) | (price_asset_new is None) | (price_asset_new == 0) | (price_benchmark_new is None) | (price_benchmark_new == 0) | (date_check is None) | (benchmark is None) | (benchmark != benchmark_check) | (date >= date_check) | (delta_performance is None):
        return "keine Daten"
    elif delta_performance > 0.05:
        return 1
    elif (delta_performance <= 0.05) and (delta_performance >= -0.05):
        return 0
    elif delta_performance < -0.05:
        return -1

df_results["1 year performance"] = df_1_year_comparison.apply(calculate_1_year_performance, axis=1)
df_1_year_comparison["Score 1 year performance"] = df_1_year_comparison.apply(calculate_1_year_performance, axis=1)

In [25]:
# Punkte Momentum 
# +1 wenn Punkte 6 Monate = +1 UND Punkte 1 Jahr = 0 oder -1, 
# 0 wenn Punkte 6 Monate = 0 ODER wenn (Punkte 6 Monate = +1 UND Punkt 1 Jahr = +1) ODER wenn (Punkte 6 Monate = -1 UND Punkte 1 Jahr = -1)
# -1 wenn Punkte 6 Monate = -1 UND Punkte 1 Jahr = 0 oder +1, 

# Voraussetzung ist, dass sowohl Punkte 6 Monate und Punkte 1 Jahr ermittelt wurden.

df_momentum = pd.DataFrame()
df_momentum[["6 months performance", "1 year performance"]] = df_results[["6 months performance", "1 year performance"]]

# Bewertungsfunktion
def calculate_momentum(row):
    new = row["6 months performance"]
    old = row["1 year performance"]
    if (new is None) | (old is None) | (new == "keine Daten") | (old == "keine Daten"):
        return "keine Daten"
    elif (new == 1) and ((old == 0) | (old == -1)):
      return 1
    elif (new == 0) | ((new == 1) and (old == 1)) | ((new == -1) and (old == -1)):
        return 0
    elif (new == -1) and ((old == 1) | (old == 0)):
        return -1

df_results["momentum"] = df_momentum.apply(calculate_momentum, axis= 1)
df_momentum["Score momentum"] = df_momentum.apply(calculate_momentum, axis= 1)

In [26]:
# Punkte für Dreimonatsreversal
# +1 Punkt wenn Performance des Assets in den vergangenen 3 1-Monats-Zeiträumen jedes Mal schlechter als der Vergleichsindex war
# -1 Punkt wenn Performance des Assets in den vergangenen 3 1-Monats-Zeiträumen jedes Mal besser als der Vergleichsindex war
# Sonst 0 Punkte

df_1to3_months_ago = pd.DataFrame()
df_latest_prices = pd.DataFrame()
df_1to3_months_tmp1 = pd.DataFrame()
df_1to3_months_tmp2 = pd.DataFrame()
df_1to3_months_comparison = pd.DataFrame()
df_1month_ago = pd.DataFrame()
df_2months_ago = pd.DataFrame()
df_3months_ago = pd.DataFrame()

# Ermittlung der aktuellsten Kurse
df_latest_prices = df_prices
df_latest_prices = df_latest_prices.sort_values("date", ascending=False)
df_latest_prices = df_latest_prices.groupby("symbol").first()
df_latest_prices["date"] = pd.to_datetime(df_latest_prices["date"])

# Ermittlung der Kurse vor 1 Monat (bzw. am darauffolgenden nächsten verfügbaren Kurs)
df_1month_ago = df_prices
date_1month = df_previous_days.loc[df_previous_days["Description"] == "vor 1 Monat", "Dates"].values[0]
date_1month = pd.to_datetime(date_1month)
df_1month_ago = df_1month_ago[df_1month_ago["date"] == date_1month]
# Überprüfung, ob Eintrag am vorliegenden Datum vorhanden ist
while df_1month_ago.empty:
    # Einen Tag zurückgehen
    date_1month -= dt.timedelta(days=1)
    df_1month_ago = df_prices[df_prices["date"] == date_1month]
df_1month_ago = df_1month_ago.drop_duplicates()
df_1month_ago = df_1month_ago.set_index("symbol")

# Ermittlung der Kurse vor 2 Monaten (bzw. am darauffolgenden nächsten verfügbaren Kurs)
df_2months_ago = df_prices
date_2months = df_previous_days.loc[df_previous_days["Description"] == "vor 2 Monaten", "Dates"].values[0]
date_2months = pd.to_datetime(date_2months)
df_2months_ago = df_2months_ago[df_2months_ago["date"] == date_2months]
# Überprüfung, ob Eintrag am vorliegenden Datum vorhanden ist
while df_2months_ago.empty:
    # Einen Tag zurückgehen
    date_2months -= dt.timedelta(days=1)
    df_2months_ago = df_prices[df_prices["date"] == date_2months]
df_2months_ago = df_2months_ago.drop_duplicates()
df_2months_ago = df_2months_ago.set_index("symbol")

# Ermittlung der Kurse vor 3 Monaten (bzw. am darauffolgenden nächsten verfügbaren Kurs)
df_3months_ago = df_prices
date_3months = df_previous_days.loc[df_previous_days["Description"] == "vor 3 Monaten", "Dates"].values[0]
date_3months = pd.to_datetime(date_3months)
df_3months_ago = df_3months_ago[df_3months_ago["date"] == date_3months]
# Überprüfung, ob Eintrag am vorliegenden Datum vorhanden ist
while df_3months_ago.empty:
    # Einen Tag zurückgehen
    date_3months -= dt.timedelta(days=1)
    df_3months_ago = df_prices[df_prices["date"] == date_3months]
df_3months_ago = df_3months_ago.drop_duplicates()
df_3months_ago = df_3months_ago.set_index("symbol")

# Zusammenführen der Dataframes
df_1to3_months_tmp1 = pd.merge(df_latest_prices, df_1month_ago, how="left", left_index=True, right_index=True, suffixes=("","_1month"))
df_1to3_months_tmp2 = pd.merge(df_1to3_months_tmp1, df_2months_ago, how="left", left_index=True, right_index=True, suffixes=("","_2months"))
df_1to3_months_comparison = pd.merge(df_1to3_months_tmp2, df_3months_ago, how="left", left_index=True, right_index=True, suffixes=("","_3months"))

# Bewertungsfunktion
def calculate_1to3_months_performance(row):
    symbol_benchmark_now = row["benchmark"]
    symbol_benchmark_1m = row["benchmark_1month"]
    symbol_benchmark_2m = row["benchmark_2months"]
    symbol_benchmark_3m = row["benchmark_3months"]
    date_now = row["date"]
    date_1m = row["date_1month"]
    date_2m = row["date_2months"]
    date_3m = row["date_3months"]
    asset_now = row["previous close"]
    asset_1m = row["previous close_1month"]
    asset_2m = row["previous close_2months"]
    asset_3m = row["previous close_3months"]
    benchmark_now = row["previous close_benchmark"]
    benchmark_1m = row["previous close_benchmark_1month"]
    benchmark_2m = row["previous close_benchmark_2months"]
    benchmark_3m = row["previous close_benchmark_3months"]
    perf_asset_now = (asset_now - asset_1m) / asset_1m
    perf_asset_1m = (asset_1m - asset_2m) / asset_2m
    perf_asset_2m = (asset_2m - asset_3m) / asset_3m
    perf_benchmark_now = (benchmark_now - benchmark_1m) / benchmark_1m
    perf_benchmark_1m = (benchmark_1m - benchmark_2m) / benchmark_2m
    perf_benchmark_2m = (benchmark_2m - benchmark_3m) / benchmark_3m
    delta_perf_now = perf_asset_now - perf_benchmark_now
    delta_perf_1m = perf_asset_1m - perf_benchmark_1m
    delta_perf_2m = perf_asset_2m - perf_benchmark_2m
    if any(x is None or pd.isnull(x) or pd.isna(x) or x == "" for x in (symbol_benchmark_now, 
                                                                        symbol_benchmark_1m, 
                                                                        symbol_benchmark_2m, 
                                                                        symbol_benchmark_3m, 
                                                                        date_now, 
                                                                        date_1m, 
                                                                        date_2m, 
                                                                        date_3m, 
                                                                        asset_now, 
                                                                        asset_1m, 
                                                                        asset_2m, 
                                                                        asset_3m, 
                                                                        benchmark_now, 
                                                                        benchmark_1m, 
                                                                        benchmark_2m, 
                                                                        benchmark_3m
                                                                        )):
        return "keine Daten"
    elif any(x > 0 for x in (delta_perf_now, delta_perf_1m, delta_perf_2m)):
        return -1
    elif any(x < 0 for x in (delta_perf_now, delta_perf_1m, delta_perf_2m)):
        return 1
    else:
        return 0

df_results["3 months reversal"] = df_1to3_months_comparison.apply(calculate_1to3_months_performance, axis= 1)
df_1to3_months_comparison["Score 3 months reversal"] = df_1to3_months_comparison.apply(calculate_1to3_months_performance, axis= 1)

In [27]:
# Punkte Gewinnwachstum
# Gewinnwachstum +1 wenn Gewinnwachstum >5%, -1 wenn Gewinnwachstum <-5%, sonst 0
# wenn EPS AJ UND EPS NJ <>0 UND EPS AJ >0, dann (EPS NJ - EPS AJ)/EPS AJ, ansonsten wenn EPS AJ vorhanden: analog, nur mit (EPS AJ-EPS VJ)/EPS VJ, sonst 0

df_eps_current_year = pd.DataFrame()
df_eps_next_year = pd.DataFrame()
df_eps_last = pd.DataFrame()
df_peg = pd.DataFrame()

# Gewinnprognosen für laufendes Geschäftsjahr und für nächstes Jahr abfragen und in 2 Dataframes abspeichern
df_eps_current_year = df_earnings_future[df_earnings_future["period"] == "0y"][["symbol", "period", "epsTrend_current"]]
df_eps_next_year = df_earnings_future[df_earnings_future["period"] == "+1y"][["symbol", "period", "epsTrend_current"]]
df_eps_current_year = df_eps_current_year.rename(columns={"epsTrend_current": "eps"})
df_eps_next_year = df_eps_next_year.rename(columns={"epsTrend_current": "eps"})

# Abfrage der EPS des vergangenen Geschäftsjahres
df_eps_last = df_fundamentals[df_fundamentals["periodType"] == "12M"][["symbol", "asOfDate", "DilutedEPS"]]
df_eps_last = df_eps_last.sort_values("asOfDate", ascending=False)
df_eps_last = df_eps_last.groupby("symbol").first()
df_eps_last = df_eps_last.rename(columns={"DilutedEPS": "eps_last"})

df_eps_current_year = df_eps_current_year.set_index("symbol")
df_eps_next_year = df_eps_next_year.set_index("symbol")

#Dataframes zusammenführen
df_peg = pd.merge(df_eps_current_year, df_eps_next_year, how="left", left_index=True, right_index=True, suffixes=("_current","_next"))
df_peg = pd.merge(df_peg, df_eps_last, how="left", left_index=True, right_index=True, suffixes=("","_last"))

def calculate_peg(row):
    now = row["period_current"]
    nxt = row["period_next"]
    last = row["asOfDate"]
    eps_now = row["eps_current"]
    eps_next = row["eps_next"]
    eps_last = row["eps_last"]
    if any(x is None or pd.isnull(x) or pd.isna(x) or x == "" for x in (now, nxt, last, eps_now, eps_next, eps_last)):
        return "keine Daten"
    elif eps_now > 0:
        if ((eps_next - eps_now)) / eps_now > 0.05:
            return 1
        elif (((eps_next - eps_now)) / eps_now <= 0.05) and (((eps_next - eps_now)) / eps_now >= -0.05):
            return 0
        elif ((eps_next - eps_now)) / eps_now < -0.05:
            return -1
    else:
        if ((eps_now - eps_last) / eps_last) > 0.05:
            return 1
        elif (((eps_now - eps_last) / eps_last) <= 0.05) and (((eps_now - eps_last) / eps_last) >= -0.05):
            return 0
        elif ((eps_now - eps_last) / eps_last) < -0.05:
            return -1
    
df_results["peg"] = df_peg.apply(calculate_peg, axis=1)
df_peg["Score PEG"] = df_peg.apply(calculate_peg, axis=1)

In [31]:
# Ergebnisspalten
result_columns = ['RoE', 
                   'EBIT margin', 
                   'equity ratio', 
                   'pe_ratio_5yr',
                   'pe_ratio_current',
                   'analysts_opinion', 
                   'earnings reaction',
                   'earnings revisions', 
                   '6 months performance', 
                   '1 year performance',
                   'momentum',
                   '3 months reversal',
                   'peg'
                   ]

# In den Ergebnisspalten NULL-Werte auffüllen
df_results[result_columns] = df_results[result_columns].fillna(0)

# Gesamptpunktzahl ermitteln
df_results["total score"] = df_results.apply(lambda row: np.sum([x for col, x in row.items() if col in result_columns and (isinstance(x, int) or isinstance(x, float))]), axis=1)

# Anzahl leerer Spalten ermitteln
df_results["criteria available"] = df_results.apply(lambda row: np.sum([1 for col, x in row.items() if col in result_columns and (isinstance(x, int) or isinstance(x, float))]), axis=1)
df_results["criteria empty"] = df_results.apply(lambda row: sum(1 for col, x in row.items() if col in result_columns and (pd.isna(x) or x == "" or x == "keine Daten" or x == "Division durch Null (EPS alt = Null)")), axis=1)

# Berechnungen zum "fairen" Preis basierend auf dem KGV und der erreichten Punktzahl:
# Funktion zur Ermittlung des KGV
def calculate_pe(row):
    price = row["previous close"]
    earnings_current = row["EPS_current"]
    earnings_past = row["EPS_last"]
    if (earnings_current is None) | (earnings_current == 0) | (earnings_current == ""):
        if (earnings_past is None) | (earnings_past == 0) | (earnings_past == ""):
            return "keine Daten"
        else:
            return (price/earnings_past)
    else: 
        return (price/earnings_current)
    
df_results["PE ratio"] = df_pe_current_results.apply(calculate_pe, axis=1)

# Maximaler Preis unter Berücksichtigung der Kriterien:
# PE maximal 15, außer:
#   Punkte mind. 5 (leere Felder zählen als 0)
#   Für jeden Punkt über 5 Punkten wird zum maximalen PE von 16 1 dazu addiert
# Basierend darauf wird der maximale Limitpreis ermittelt anhand der EPS des laufenden Jahres (falls vorhanden), ansonsten anhand des Vorjahres
df_pe_fair_combined = pd.DataFrame()
df_total_score = pd.DataFrame()
df_pe_fair = pd.DataFrame()
df_pe_fair = pd.DataFrame(df_pe_current_results)
df_pe_future = pd.DataFrame()
df_pe_future = pd.DataFrame(df_earnings_future[df_earnings_future["period"] == "0y"][["epsTrend_current", "symbol"]])
df_pe_future = df_pe_future.set_index("symbol")
df_total_score = pd.DataFrame(df_results["total score"])
df_pe_fair_combined = pd.merge(df_pe_fair, df_pe_future, how="left", left_index=True, right_index=True)
df_pe_fair_combined = pd.merge(df_pe_fair, df_total_score, how="left", left_index=True, right_index=True)

def calculate_max_pe(row):
    multiple_base = row["total score"]
    if (multiple_base is None) or (multiple_base == 0) or (multiple_base == ""):
        return "keine Daten"
    else:
        if multiple_base <= 5:
            return 16
        elif (multiple_base > 5) and (multiple_base <= 13):
            return (16 + multiple_base - 5)
        else:
            return "Fehler"

def calculate_price_max_current(row):
    eps_now = row["EPS_current"]
    eps_old = row["EPS_last"]
    multiple = row["total score"]
    if (eps_now is None) or (eps_now == 0) or (eps_now == "") or (multiple == "Fehler"):
        if (eps_old is None) or (eps_old == 0) or (eps_old == "") or (multiple == "Fehler"):
            return "keine Daten"
        else:
            return eps_old
    else:
        return eps_now

def calculate_price_max_future(row):
    eps = row["EPS_current"]
    multiple = row["total score"]
    if (eps is None) or (eps == 0) or (eps == "") or (multiple == "Fehler"):
        return "keine Daten"
    else:
        return eps

df_results["PE ratio (max) for order"] = df_pe_fair_combined.apply(calculate_max_pe, axis=1)
df_results["price_component_1"] = df_pe_fair_combined.apply(calculate_max_pe, axis=1)
df_results["price_component_2"] = df_pe_fair_combined.apply(calculate_price_max_current, axis=1)
df_results["future_price_component_2"] = df_pe_fair_combined.apply(calculate_price_max_future, axis=1)

# Konvertieren in float-Datentyp
df_results["price_component_1"] = df_results["price_component_1"].replace(["keine Daten", "Fehler"], float("NaN"))
df_results["price_component_2"] = df_results["price_component_2"].replace(["keine Daten", "Fehler"], float("NaN"))
df_results["future_price_component_2"] = df_results["price_component_2"].replace("keine Daten", float("NaN"))
df_results["price_component_1"] = df_results["price_component_1"].astype(float)
df_results["price_component_2"] = df_results["price_component_2"].astype(float)
df_results["future_price_component_2"] = df_results["price_component_2"].astype(float)

# Ergebnisse erzeugen
df_results["price (max) for order"] = df_results.apply(lambda row: row["price_component_1"] * row["price_component_2"], axis=1)
df_results["future price (max) for order"] = df_results.apply(lambda row: row["price_component_1"] * row["future_price_component_2"], axis=1)
df_results = df_results.drop(["price_component_1", "price_component_2", "future_price_component_2"], axis=1)

In [29]:
list_sheets = [df_analysts, 
               df_ebit_latest, 
               df_pe_5_results, 
               df_pe_current_results, 
               df_history_announcements, 
               df_revisions, 
               df_6_months_comparison, 
               df_1_year_comparison, 
               df_momentum, 
               df_1to3_months_comparison, 
               df_peg
               ]

list_master_data = [df_prices, 
                    df_fundamentals, 
                    df_fundamentals_current, 
                    df_earnings, 
                    df_earnings_future
                    ]

dict_sheets = {"Analystenmeinung": df_analysts, 
               "EBIT": df_ebit_latest, 
               "5-Jahres-KGV": df_pe_5_results, 
               "KGV aktuell": df_pe_current_results, 
               "Reaktion auf Quartalszahlen": df_history_announcements, 
               "Gewinnrevisionen": df_revisions, 
               "6-Monats-Performance": df_6_months_comparison, 
               "1-Jahres-Performance": df_1_year_comparison, 
               "Kursmomentum": df_momentum, 
               "3-Monats_Reversal": df_1to3_months_comparison, 
               "Gewinnwachstum": df_peg
               }
dict_master_data = {"Kurse": df_prices, 
                    "Fundamental-KPIs": df_fundamentals, 
                    "Fundamental-KPIs_aktuell": df_fundamentals_current, 
                    "Quartalszahlen": df_earnings, 
                    "Quartalszahlen Prognose": df_earnings_future
                    } 

In [30]:
# Erstellen eines Styler-Objekts für die Formatierung
def format_sheet(sheet):
    # Hintergrundfarbe für Überschriften
    header_fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
    # Spaltenüberschriften fett formatieren
    for cell in sheet[1]:
        cell.font = cell.font.copy(bold=True)
    # Hintergrundfarbe für Überschriften anwenden
    for cell in sheet[1]:
        cell.fill = header_fill
    # Spaltenbreite anpassen
    for column in sheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in sheet[column_letter]:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2  # 1.2 entspricht ungefähr 1 Pixel pro Zeichen
        sheet.column_dimensions[column_letter].width = adjusted_width
    return sheet

# Ergebnisse in Excel-Datei schreiben
output_path = os.path.dirname(input_data)
output_file = "Ergebnisse Bewertung_per " + dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".xlsx"
output_data = os.path.join(output_path, output_file)

# Sortieren des Dataframes "df_results" nach dem Index

merged_results = pd.merge(df_input_data, df_results, left_on="Symbol (Yahoo)", right_index=True, how="left")

# Ergebnisdatei erstellen
with pd.ExcelWriter(output_data, engine='openpyxl') as my_writer:
    # Inputdaten aus der Quellexceldatei in Ergebnisdatei kopieren und die Formatierung beibehalten
    merged_results.to_excel(my_writer, sheet_name="Ergebnisse", index=False)
    workbook = my_writer.book
    worksheet = workbook["Ergebnisse"]
    format_sheet(worksheet)
    worksheet.auto_filter.ref = worksheet.dimensions
    # Weitere Tabellenblätter formatieren
    for key, value in dict_sheets.items():
        value.to_excel(my_writer, sheet_name=key, index=False)
        worksheet = workbook[key]
        format_sheet(worksheet)
        worksheet.auto_filter.ref = worksheet.dimensions
    workbook.save(output_data)

# Endzeitpunkt erfassen
end_time = time.time()

# Gesamtdauer berechnen
execution_time = end_time - start_time

print(f"Die Ergebnisse wurden in der Datei {output_file} im Verzeichnis {output_path} gespeichert.")
print(f"Die Laufzeit des Skripts betrug {execution_time:.2f} Sekunden.")

C:\Users\HannesDuerr\AppData\Local\Temp\ipykernel_1828\1877222745.py:7: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)


Die Ergebnisse wurden in der Datei Ergebnisse Bewertung_per 2023-08-18_14-56-27.xlsx im Verzeichnis C:/Users/HannesDuerr/OneDrive - Deutsche Bahn/Data Science gespeichert.
Die Laufzeit des Skripts betrug 589.40 Sekunden.
